In [213]:
import numpy as np
import pandas as pd
import math

In [214]:
from datetime import datetime
todays_date = datetime.today().strftime('%m/%d/%Y')

In [215]:
pilot_or_exp = 1 # 1 pilot, 2 experimental

In [216]:
# JUST WANT ITERATIONS prolific_multiple_subjects_2, prolific_multiple_subjects_3, 
# and non_prolific (filtered)

In [217]:
template_path = '/Users/chrisiyer/_Current/thesis_folder/face_experiment/results/demographics_template.csv'
datapath = '/Users/chrisiyer/_Current/thesis_folder/face_experiment/results/data/classification/demographics/'
datafile = 'summarydata_faces_classification_task_02_04_2022.csv'

dempath = '/Users/chrisiyer/_Current/thesis_folder/face_experiment/results/data/classification/demographics/'
demfile1 = 'prolific_multiple_subjects_2.csv'
demfile2 = 'prolific_multiple_subjects3_nonprolific.csv'
outputfile = dempath + 'classification_nonpro_redcap.csv'

df = pd.read_csv(datapath + datafile)
df_exp = df[df.trial_type == 'demographics'] 
# df_exp = pd.concat([
#     df_exp[df_exp.iteration == 'prolific_multiple_subjects_2'], 
#     df_exp[df_exp.iteration == 'prolific_multiple_subjects_3']
#     ])
df_exp = df_exp[df_exp.iteration == 'non_prolific'].iloc[2:,:]
df_pro = pd.concat([pd.read_csv(dempath + demfile1), pd.read_csv(dempath + demfile2)]) # need to coombine two files for this run

output = pd.read_csv(template_path)#.iloc[:,0:output.shape[1]-1]

In [218]:
def create_row(record_id, race, hispanic, gender, other_gender, age, date):
    # race: ['1', '2', '3', '4', '5', '6', '99'] or '' at each
    # hispanic: 0 for no, 1 for yes
    # gender: 0 for male, 1 for female, 2 for other
    # other gender: text
    # age is just a number
    # date: of experimental completion
    
    row = {
        "record_id": record_id,
        "dateofentry": todays_date,
        "experimenter_name": 1, # tyler bonnen
        "irb_num": 2, # 10944
        "grant_sub": 99, # other
        "other_grant": "Anderson grant to tyler bonnen",

        "sub_gender": gender, #0 male 1 female 2 other
        "other_gender": other_gender,
        "sub_race___1": race[0], #native
        "sub_race___2": race[1], #asian
        "sub_race___3": race[2], #pacific islander
        "sub_race___4": race[3], #black
        "sub_race___5": race[4], #white
        "sub_race___6": race[5], #latinx
        "sub_race___99": race[6], #other
        "sub_hispanic": hispanic, #0 no 1 yes
        "sub_age": age, 
        "data_collection_date": date,
        "data_type": pilot_or_exp,
        "modality___1": 1, # behavioral
        "modality___2": "",
        "modality___3": "",
        "modality___4": "",
        "modality___5": "",
        "modality___6": "",
        "modality___7": "",
        "modality___8": "",
        "modality_other": "",
        "collected_online": 1, #yes
        "online_platform___1": 0, #mturk
        "online_platform___2": 1, #prolific,
        "online_platform___3": 0,
        "online_platform___4": 0,
        "other_online": "",
        "participant_exclusion": 0, #no
        "why_excluded": "",
        "participant_withdraw": 0, #no
        "why_withdraw": "",
        "comments": "",
        "date_renewal": "",
        "renewal_report": "",
        "demographics_complete": 0, #incomplete, I guess?
        "experimenter": "",
        "protocol": "",
        "date": "",
        "description": "",
        "report_required": "",
        "why_no_report": "",
        "date_form_submitted": "",
        "date_form_approved": "",
        "date_reported_renewal": "",
        "adverse_events_complete": ""
    }
    return row

In [219]:
def get_race(race_raw):
    output = [0]*7
    if 'Native' in race_raw: output[0] = '1'
    if 'Asian' in race_raw: output[1] = '1'
    if 'Pacific Islander' in race_raw: output[2] = '1'
    if 'Black' in race_raw: output[3] = '1'
    if 'White' in race_raw: output[4] = '1'
    if 'Latinx' in race_raw: output[5] = '1'
    if 'Other' in race_raw: output[6] = '1'
    return output
    
def get_gender(gender_raw): 
    if (gender_raw.startswith('F') or gender_raw.startswith('f')):
        return 1
    elif (gender_raw.startswith('M') or gender_raw.startswith('m')):
        return 0
    else:
        return 2

In [220]:
for i in range(len(df_exp)):
    # for each record, add a row with necessary information
    sub_id = df_exp['subject_id'].tolist()[i]
    pro_row = df_pro[df_pro['participant_id'] == sub_id]
    
    if pro_row.status.iloc[0] == 'APPROVED':
        
        # first, get race, gender
        race_raw = df_exp.iloc[i,:]['race']
        hisp_text = df_exp.iloc[i,:]['latinx']
        if hisp_text == "No": hispanic = 0
        else: hispanic = 1
        gender_raw = df_exp.iloc[i,:]['gender']
        if gender_raw == 'other':
            gender_raw = df_exp.iloc[i,:]['gender_other']


        if not isinstance(gender_raw, str): need_gender = True
        else: need_gender = False

        sub_id = df_exp.iloc[i,:]['_id']


    #     # USE THIS FOR FULL FORM
    #     age = df_exp.iloc[i,:]['age']
    #     date_time = df_exp.iloc[i,:]['datetime']
    #     date = datetime.strptime(date_time[0:date_time.index(',')], '%m/%d/%Y').strftime('%m/%d/%Y')

        # USE THIS NORMALLY
        # just need age and date from df_pro, maybe gender

        age = pro_row.age
        date = pro_row.completed_date_time.iloc[0].split(" ")[0]
        if need_gender: gender_raw = pro_row['Sex'].iloc[0]


        row = create_row(
            record_id = i,
            race=get_race(race_raw), 
            hispanic=hispanic, 
            gender=get_gender(gender_raw), 
            other_gender = "", 
            age = age, 
            date = date)
        output.loc[len(output)] = row
    
output

,record_id,dateofentry,experimenter_name,irb_num,grant_sub,other_grant,sub_gender,other_gender,sub_race___1,sub_race___2,...,protocol,date,description,report_required,why_no_report,date_form_submitted,date_form_approved,date_reported_renewal,adverse_events_complete,Unnamed: 52
0,0,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
1,1,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,1,...,,,,,,,,,,NaN
2,2,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
3,3,04/03/2022,1,2,99,Anderson grant to tyler bonnen,0,,1,0,...,,,,,,,,,,NaN
4,4,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,1,...,,,,,,,,,,NaN
5,5,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
6,6,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,1,...,,,,,,,,,,NaN
7,7,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,1,...,,,,,,,,,,NaN
8,8,04/03/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
9,9,04/03/2022,1,2,99,Anderson grant to tyler bonnen,0,,0,0,...,,,,,,,,,,NaN


In [221]:
output.iloc[:,0:output.shape[1]-1].to_csv(outputfile, index=False)